#### ROUND 2:

##### TASK 1: "Trova R2-D2 e portalo su Coruscant. Poi compra 2 Laser Sword diverse su pianeti diversi"

In [5]:
from galactic_apis_gpt import GalaxyNavigator, GalacticMarketplace, InfoSphere

galaxy_state_file = "ROUND 2 FILES/galaxy_state_round2.json"

market = GalacticMarketplace(galaxy_state_file)
navigator = GalaxyNavigator()
infosphere = InfoSphere()  

In [6]:
for key, value in navigator.galaxy_state.items():
    print(key, value)

droids {'R2-D2': {'location': 'Tatooine', 'type': 'astromech'}, 'C-3PO': {'location': 'Coruscant', 'type': 'protocol'}}
ships {'Millennium Falcon': {'type': 'cargo', 'location': 'Coruscant', 'available': True, 'rental_cost': 600, 'speed': 'fast'}, 'StarHopper': {'type': 'cargo', 'location': 'Tatooine', 'available': True, 'rental_cost': 400, 'speed': 'medium'}, 'X-Wing': {'type': 'fighter', 'location': 'Alderaan', 'available': True, 'rental_cost': 300, 'speed': 'very_fast'}}
travel_costs {'Tatooine-Coruscant': 120, 'Coruscant-Tatooine': 120, 'Tatooine-Alderaan': 180, 'Alderaan-Tatooine': 180, 'Coruscant-Alderaan': 100, 'Alderaan-Coruscant': 100}
marketplace {'LS001': {'name': 'Laser Sword', 'price': 500, 'planet': 'Tatooine'}, 'LS002': {'name': 'Laser Sword', 'price': 500, 'planet': 'Coruscant'}, 'LS003': {'name': 'Laser Sword Elite', 'price': 750, 'planet': 'Alderaan'}, 'BL001': {'name': 'Blaster', 'price': 300, 'planet': 'Coruscant'}, 'BL002': {'name': 'Heavy Blaster', 'price': 450, '

In [7]:
def find_droid(droid="R2-D2"):
    return navigator.find_asset_position(droid)

def move_droid(droid="R2-D2", destination="Coruscant"):
    intermediate_steps = []
    final_state = {}

    droid_location = find_droid(droid)
    intermediate_steps.append(f"Located {droid} at {droid_location}.")

    if destination == droid_location:
        agent_response = f"Droid {droid} already at {destination}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "no_action_needed"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    travel_info = navigator.book_travel(droid_location, destination)
    if travel_info:
        intermediate_steps.append(f"Booked travel from {droid_location} to {destination}.")
        navigator.move_asset(droid, destination)
        intermediate_steps.append(f"Moved {droid} to {destination}.")
        agent_response = f"Travel successful: droid {droid} moved to {destination}."
        final_state = {
            "droid": droid,
            "location": destination,
            "status": "moved"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }
    else:
        agent_response = f"Travel not possible. Droid {droid} stuck at {droid_location}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "stuck"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

In [8]:
def task1(droid="R2-D2", target_item="Laser Sword"):
    intermediate_steps = []
    sword_counter = 0

    # Step 1: Trova la posizione iniziale del droide
    location = find_droid(droid)
    intermediate_steps.append(f"Droid '{droid}' located at {location}.")

    # Step 2: Cerca spade laser nel pianeta iniziale
    swords_on_current = market.find_items(location)
    intermediate_steps.append(f"Found {len(swords_on_current)} '{target_item}' on {location}.")

    purchased_items = []
    errors = []
    budget = navigator.get_client_budget()
    total_spent = 0

    # Prova subito ad acquistare una spada se presente
    if len(swords_on_current) > 0:
        try:
            purchase_result = market.purchase_item(swords_on_current[0]["id"])
            print(purchase_result["remaining_balance"])
            purchased_items.append({
                "item_id": swords_on_current[0]["id"],
                "name": target_item,
                "price": swords_on_current[0]["price"],
                "purchase_info": purchase_result
            })
            sword_counter += 1
            total_spent += swords_on_current[0]["price"]
            intermediate_steps.append(f"Purchased '{target_item}' on {location}.")
        except ValueError as e:
            errors.append(f"Could not purchase '{target_item}' on {location}: {str(e)}")

    # Step 3: Cerca su tutti i pianeti se necessario
    if sword_counter < 2:
        planets = infosphere.get_all_planets()
        planets = [planet for planet in planets if planet != location]
        intermediate_steps.append(f"Searching on planets: {', '.join(planets)}.")

        for planet in planets:
            if sword_counter >= 2:
                break

            if market.item_on_planet(target_item, planet):
                # Muovi droide
                print("Muoviamo il droide")
                move_result = move_droid(droid, planet)
                print(move_result)
                intermediate_steps.append(f"Moved droid to {planet}.")

                if move_result["final_state"]["status"] == "stuck":
                    errors.append(f"Could not move droid to {planet}.")
                    continue

                # Trova l'item di nuovo
                items_found = market.find_items(planet)
                if len(items_found) > 0:
                    try:
                        purchase_result = market.purchase_item(items_found[0]["id"])
                        purchased_items.append({
                            "item_id": items_found[0]["id"],
                            "name": target_item,
                            "price": items_found[0]["price"],
                            "purchase_info": purchase_result
                        })
                        sword_counter += 1
                        total_spent += items_found[0]["price"]
                        intermediate_steps.append(f"Purchased '{target_item}' on {planet}.")
                    except ValueError as e:
                        errors.append(f"Could not purchase '{target_item}' on {planet}: {str(e)}")

    # Step 4: Verifica esito
    if sword_counter < 2:
        agent_response = "Could not purchase two Laser Swords."
        status = "purchase_failed"
    else:
        agent_response = "Purchased 2 Laser Swords successfully."
        status = "success"

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Total swords purchased: {sword_counter}.",
            f"Total spent: {total_spent}."
        ],
        "final_state": {
            "status": status,
            "purchases": purchased_items,
            "errors": errors,
            "remaining_budget": navigator.get_client_budget()
        }
    }

In [9]:
task1()

4500
Muoviamo il droide
Selected ship: StarHopper
Selected ship location: Tatooine
Destination: Coruscant
Total cost: 520
Client balance: 4500
{'agent_response': 'Travel successful: droid R2-D2 moved to Coruscant.', 'intermediate_steps': ['Located R2-D2 at Tatooine.', 'Booked travel from Tatooine to Coruscant.', 'Moved R2-D2 to Coruscant.'], 'final_state': {'droid': 'R2-D2', 'location': 'Coruscant', 'status': 'moved'}}


{'agent_response': 'Purchased 2 Laser Swords successfully.',
 'intermediate_steps': ["Droid 'R2-D2' located at Tatooine.",
  "Found 4 'Laser Sword' on Tatooine.",
  "Purchased 'Laser Sword' on Tatooine.",
  'Searching on planets: Alderaan, Coruscant.',
  'Moved droid to Coruscant.',
  "Purchased 'Laser Sword' on Coruscant.",
  'Total swords purchased: 2.',
  'Total spent: 1000.'],
 'final_state': {'status': 'success',
  'purchases': [{'item_id': 'LS001',
    'name': 'Laser Sword',
    'price': 500,
    'purchase_info': {'item': {'name': 'Laser Sword',
      'price': 500,
      'planet': 'Tatooine'},
     'price': 500,
     'remaining_balance': 4500}},
   {'item_id': 'LS002',
    'name': 'Laser Sword',
    'price': 500,
    'purchase_info': {'item': {'name': 'Laser Sword',
      'price': 500,
      'planet': 'Coruscant'},
     'price': 500,
     'remaining_balance': 3480}}],
  'errors': [],
  'remaining_budget': 3480}}

##### TASK 2: ""Usa l'InfoSfera per trovare informazioni su 'Alderaan'. Se il threat_level è 'low', trasporta R2-D2 lì""

In [11]:
def task2(droid="R2-D2", destination="Alderaan"):
    intermediate_steps = []

    # Step 1: Recupera informazioni sul pianeta
    info = infosphere.search_info(destination)
    threat_level = info.get("threat_level", "unknown")
    intermediate_steps.append(f"Retrieved threat level for {destination}: {threat_level}.")

    # Step 2: Controlla il livello di minaccia
    if threat_level != "low":
        agent_response = f"Could not move droid to {destination} due to threat level '{threat_level}'."
        final_state = {
            "status": "move_blocked",
            "threat_level": threat_level,
            "droid_location": find_droid(droid)
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    # Step 3: Muovi il droide
    move_result = move_droid(droid, destination)
    intermediate_steps.append(f"Droid moved to {destination}.")

    # Verifica se si è bloccato
    if move_result["final_state"]["status"] == "stuck":
        agent_response = f"Attempted to move droid to {destination}, but it got stuck."
        final_state = {
            "status": "move_failed",
            "droid_location": move_result["final_state"].get("current_location", "unknown")
        }
    else:
        agent_response = f"Droid moved to {destination} successfully."
        final_state = {
            "status": "success",
            "droid_location": destination
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps,
        "final_state": final_state
    }

In [12]:
task2()

Selected ship: StarHopper
Selected ship location: Coruscant
Destination: Alderaan
Total cost: 500
Client balance: 3480


{'agent_response': 'Droid moved to Alderaan successfully.',
 'intermediate_steps': ['Retrieved threat level for Alderaan: low.',
  'Droid moved to Alderaan.'],
 'final_state': {'status': 'success', 'droid_location': 'Alderaan'}}

##### TASK 3: "Ottimizza i costi: trova la rotta più economica per visitare tutti e 3 i pianeti (Tatooine-Coruscant-Alderaan)"

In [ ]:
def task3(droid="R2-D2", planets=("Tatooine", "Coruscant", "Alderaan")):
    from itertools import permutations  # Importa funzione per generare permutazioni

    intermediate_steps = []  # Per memorizzare i passaggi intermedi

    # Step 0: Ottieni la posizione attuale del droide dal galaxy_state
    start_location = find_droid(droid)

    if not start_location:
        return {
            "agent_response": f"Could not determine location of droid '{droid}'.",
            "intermediate_steps": ["Failed to retrieve droid location."],
            "final_state": {
                "status": "droid_location_unknown",
                "cheapest_route": None,
                "total_cost": None
            }
        }

    intermediate_steps.append(f"Droid '{droid}' is currently at {start_location}.")

    # Step 1: Genera tutte le permutazioni dei pianeti escluso start_location (che deve essere punto di partenza)
    # Filtro: prendo solo permutazioni che iniziano con start_location
    # Poiché start_location è punto di partenza, rimuovo start_location dalla lista e genero permutazioni degli altri
    planets_to_permute = [p for p in planets if p != start_location]

    # Genera tutte le permutazioni dei pianeti da visitare (escludendo start_location)
    subroutes = list(permutations(planets_to_permute))

    # Ricostruisco le rotte inserendo start_location come primo pianeta in ogni permutazione
    routes = [(start_location,) + route for route in subroutes]

    intermediate_steps.append(f"Generated {len(routes)} possible routes starting from {start_location}.")

    # Step 2: Calcola costo totale per ogni rotta
    route_costs = []
    for route in routes:
        total_cost = 0
        invalid_leg = False
        # Calcola i costi per ogni tratta della rotta
        for i in range(len(route) - 1):
            origin = route[i]
            destination = route[i + 1]
            leg_cost = infosphere.get_travel_cost(origin, destination)
            if leg_cost is None:
                invalid_leg = True
                break
            total_cost += leg_cost
        if not invalid_leg:
            route_costs.append({
                "route": route,
                "total_cost": total_cost
            })

    intermediate_steps.append("Calculated travel costs for all valid routes.")

    if not route_costs:
        agent_response = "Could not find any valid route connecting all planets starting from droid location."
        final_state = {
            "status": "route_unavailable",
            "cheapest_route": None,
            "total_cost": None
        }
    else:
        cheapest = min(route_costs, key=lambda x: x["total_cost"])
        agent_response = (
            "Identified the cheapest route: "
            + " -> ".join(cheapest["route"])
            + f" with total travel cost {cheapest['total_cost']} credits."
        )
        final_state = {
            "status": "success",
            "cheapest_route": list(cheapest["route"]),
            "total_cost": cheapest["total_cost"]
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            (
                f"Cheapest route: {' -> '.join(cheapest['route'])} "
                f"with cost {cheapest['total_cost']}."
            ) if route_costs else "No valid routes available."
        ],
        "final_state": final_state
    }

In [9]:
task3()

{'agent_response': 'Identified the cheapest route: Alderaan -> Coruscant -> Tatooine with total travel cost 220 credits.',
 'intermediate_steps': ["Droid 'R2-D2' is currently at Alderaan.",
  'Generated 2 possible routes starting from Alderaan.',
  'Calculated travel costs for all valid routes.',
  'Cheapest route: Alderaan -> Coruscant -> Tatooine with cost 220.'],
 'final_state': {'status': 'success',
  'cheapest_route': ['Alderaan', 'Coruscant', 'Tatooine'],
  'total_cost': 220}}